In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.read_csv('/content/sample_data/Churn_Modelling.csv')
dataset.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [ ]:
x = dataset.iloc[:, 3:13]
y = dataset.iloc[:,-1]

In [ ]:
geography = pd.get_dummies(x.Geography, drop_first=True)
gender = pd.get_dummies(x.Gender, drop_first=True)

In [ ]:
x = pd.concat([x, geography, gender], axis=1)
x.drop(['Geography','Gender'], axis=1, inplace=True)
x.tail()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1
9999,792,28,4,130142.79,1,1,0,38190.78,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)



In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import ReLU,LeakyReLU,ELU,Dropout


In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu', input_dim=11))
classifier.add(Dense(6, kernel_initializer='he_uniform', activation='relu'))
classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))

classifier.compile(optimizer='Adamax', loss='binary_crossentropy', metrics=['accuracy'])

model_history = classifier.fit(x_train,y_train,validation_split=0.3, batch_size=10, epochs= 100)

Epoch 1/100
560/560 [==============================] - 15s 2ms/step - loss: 0.6699 - accuracy: 0.7057 - val_loss: 0.5752 - val_accuracy: 0.7663
Epoch 2/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5751 - accuracy: 0.7614 - val_loss: 0.5224 - val_accuracy: 0.7904
Epoch 3/100
560/560 [==============================] - 1s 2ms/step - loss: 0.5232 - accuracy: 0.7841 - val_loss: 0.4939 - val_accuracy: 0.7933
Epoch 4/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4926 - accuracy: 0.7972 - val_loss: 0.4767 - val_accuracy: 0.7983
Epoch 5/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4880 - accuracy: 0.7847 - val_loss: 0.4655 - val_accuracy: 0.7983
Epoch 6/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4653 - accuracy: 0.7978 - val_loss: 0.4575 - val_accuracy: 0.7987
Epoch 7/100
560/560 [==============================] - 1s 2ms/step - loss: 0.4607 - accuracy: 0.7924 - val_loss: 0.4514 - val_accuracy: 0.798

In [ ]:
y_pred = classifier.predict(x_test)
y_pred = y_pred>0.5

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)

from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, y_pred)

print(cm, score)

[[1534   73]
 [ 213  180]] 0.857


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [33]:
def create_model(layers, activation):
  model = Sequential()
  for i,nodes in enumerate(layers):
    if i==0:
      model.add(Dense(nodes, input_dim=x_train.shape[1], activation=activation))
      model.add(Dropout(0.3))
    else:
      model.add(Dense(nodes, activation=activation))
      model.add(Dropout=0.3)
  model.add(Dense(1,kernel_initializer='glorot_uniform', activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [34]:
model= KerasClassifier(build_fn=create_model, verbose=0)

In [35]:
layers=[[20],[40,20],[45,30,15]]
activations=['sigmoid', 'relu']
param_grid = dict(layers=layers, activation = activations, batch_size=[128,256], epochs=[30])
grid = GridSearchCV(estimator = model,  param_grid= param_grid, cv=5)

In [36]:
grid_result = grid.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: add() got an unexpected keyword argument 'Dropout'

  FitFailedWarning)


In [37]:
[grid_result.best_score_, grid_result.best_params_]

[0.8462499856948853,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [20]}]